### Imprts and functions

In [ ]:
import fbprophet
import pickle
import pandas as pd
import folium
from shapely.geometry import Point
from folium.plugins import HeatMap
from shapely.geometry.polygon import Polygon
import pandas as pd
import numpy as np
import json
import branca.colormap as cm
import matplotlib.pyplot as plt
from datetime import timedelta
import datetime
import requests
import folium
from folium import plugins
from scipy.ndimage import imread
from folium import FeatureGroup, LayerControl, Map, Marker

In [ ]:
def make_forecast(leave_month, leave_day, leave_hour, leave_min = 0, window = 6):
    '''
    Given the month, day, and hour, minutes and the window of time (all ints),
    create a forecast that will be used to build the map. It is assumed to be 2018,
    since that is how the models are built
        
    :leave_month: an int, 1-12, for the month of prediction
    :leave_day: an int, 1-31, for the day of prediction
    :leave_hour: an int, 0-23, for the hour of prediction
    :leave_min: an int, 0-60, for the minute of prediction
    :window: an int, for the window of time for forecasts

    Returns dataframe of forecasted results
    '''
    
    leave_time = datetime.datetime(2018, leave_month, leave_day,leave_hour , leave_min)
    before = [leave_time - i* timedelta(minutes = 10) for i in range(2,window+1)]
    after = [leave_time + i* timedelta(minutes = 10) for i in range(2,window+1)]
    
    before.append(leave_time)
    before = before+after
    #future = pd.DataFrame(before, columns= ['ds'])
    
    return pd.DataFrame(before, columns= ['ds'])

In [ ]:
def here_api_call(start_lat, start_long, finish_lat, finish_long, mode = 'shortest'):
    '''
    Given the start and end lat and long, and the desired mode, uses the 
    Here API to create a route
        
    :start_lat: the starting latitude
    :start_long: the starting longitude 
    :finish_lat: the end latitude
    :finish_long: the end longitude
    :mode: mode, either fastest or shortest

    Returns dataframe of forecasted results
    '''   
    if mode == 'fastest':
        url = 'https://route.api.here.com/routing/7.2/calculateroute.json?app_id=XXX&app_code=CHFq-4oV3pch-gneEAGOEA&waypoint0=geo!%s,%s&waypoint1=geo!%s,%s&mode=fastest;car;traffic:disabled;motorway:-2'\
        %(start_lat,start_long,finish_lat,finish_long)
    if mode == 'shortest':
            url = 'https://route.api.here.com/routing/7.2/calculateroute.json?app_id=XXX&app_code=CHFq-4oV3pch-gneEAGOEA&waypoint0=geo!%s,%s&waypoint1=geo!%s,%s&mode=shortest;car;traffic:disabled'\
            %(start_lat,start_long,finish_lat,finish_long)
    
    return requests.get(url).json()['response']['route'][0]['leg'][0]

def directions(start_lat=41.9925487,start_long=-87.6560866,finish_lat=41.8762291,finish_long=-87.6553928, mode= 'shortest'):
    '''
    Given the start and end lat and long, and the desired mode, uses the 
    Here API to map route
        
    :start_lat: the starting latitude
    :start_long: the starting longitude 
    :finish_lat: the end latitude
    :finish_long: the end longitude
    :mode: mode, either fastest or shortest

    Returns dataframe of forecasted results
    '''   
    route = here_api_call(start_lat, start_long, finish_lat, finish_long)
    lat = []
    long = []
    calc  = 0
    for route in route['maneuver']:
        lat.append(route['position']['latitude'])
        long.append(route['position']['longitude'])
        calc += route['length'] #meters
        #print(route['length']*0.000621371) #this is meters to miles, need miles per hour (/mph)
    coords = list(zip(lat,long))
    #print(calc*0.000621371) #meters to miles
    return coords

def create_map(mapping_df, start_lat=41.9925487,start_long=-87.6560866,finish_lat=41.8762291,finish_long=-87.6553928, mode= 'shortest'):
    '''
    Given a dataframe of predicted values, the start and end lat and long, 
    and the desired mode, uses the Here API to map route
        
    :mapping_df: forecasted values
    :start_lat: the starting latitude
    :start_long: the starting longitude 
    :finish_lat: the end latitude
    :finish_long: the end longitude
    :mode: mode, either fastest or shortest

    returns map
    '''  
    
    #traffic region data
    coords_df = pd.read_pickle('coords_df.pkl')
    
    #create map, centered on Chicago
    m = folium.Map(location=[41.8781, -87.6298],
                     zoom_start = 11)
    
    #add tiles
    folium.TileLayer(tiles='OpenStreetMap').add_to(m)
    
    #get directions
    coords = directions(start_lat, start_long, finish_lat, finish_long, mode)
    
    #loop through directions
    for index, item in enumerate(coords):
        #add a pop up for the starting location
        if index == 0:
            folium.Marker(item, popup='Start').add_to(m)

        #add directions
        try:
            point = item,coords[index+1]
            folium.PolyLine(point, color= 'black').add_to(m)

        #when you hit the end of the direction, add that as the end location
        except IndexError:
            folium.Marker(item, popup='Finish').add_to(m)

    #loop through dataframe
    for i in range(len(mapping_df)):
        
        map_row = mapping_df.iloc[i:i+1].transpose()[1:]
        
        #assign color to each spead
        colormap = cm.LinearColormap(
            ['red', 'yellow', 'green'],
            vmin= 17, 
            vmax= 35)
        
        #add each window of time as a layer
        feature_group = FeatureGroup(name = mapping_df.ds[i], overlay=False)

        #each region needs to be overlayed
        for regions in range(1,30):

            NW = coords_df.iloc[0,regions]
            NE = coords_df.iloc[1,regions]
            SE = coords_df.iloc[2,regions]
            SW = coords_df.iloc[3,regions]


            point = [NW,NE,SE,SW]


            folium.PolyLine(point,fill='black', popup='%s'%(float(map_row.iloc[regions-1])),
                            fill_color=colormap(float(map_row.iloc[regions-1])), fill_opacity = .5, weight=0).add_to(feature_group)

        #add this all together
        m.add_child(feature_group)
    
    #allows control over all the layers added
    m.add_child(folium.map.LayerControl())
    
    #return the map
    return m

In [ ]:
def forecast_map(leave_month, leave_day, leave_hour, leave_min = 0, window = 6,start_lat=41.9925487,start_long=-87.6560866,finish_lat=41.8762291,finish_long=-87.6553928, mode= 'shortest'):
    future =  make_forecast(leave_month, leave_day, leave_hour, leave_min, window)
    mapping_df = future

    for region in range(1,30):
        globals()['forecast%s'%(region)] = globals()['model%s'%(region)].predict(future)

        colum_name = 'region%s'%(region)
        mapping_df[colum_name] = globals()['forecast%s'%(region)].yhat

    m = create_map(mapping_df)
    
    return mapping_df, m

### Load Data

In [ ]:
for i in range(1,30):
    model_name = 'region%s_model'%(i)
    metric_name = 'region%s_stats'%(i)
    forecast_name = 'region%s_forecast'%(i)

    with open(model_name, "rb") as f:
        globals()['model%s'%(i)] = pickle.load(f)

    with open(metric_name, "rb") as g:
        metrics = pickle.load(g)
    
    with open(forecast_name, "rb") as h:
        globals()['forecast%s'%(i)] = pickle.load(h)

### Make Predictions